# L1.4 Классы, декораторы, ООП, stdlib

* Мы подкрадываемся к полному покрытию всех тем, которые я хотел рассказать непосредственно про синтаксис языка Python
* Дальше будем больше про библиотеки и анализ. Начинайте вспоминать статистику например

## Classes

Для наших целей классы это такая штука в которую можно спрятать какую-то логику, причём так чтобы это.

Вообще, все ваши классы лучше наследовать от общего типа `object`. Но так как мы пока даже не обсудили, что такое наследование, то давайте просто запомним что классы лучше определять вот так:

```python
class MyClass(object):
    ...
```

При этом класс это определение некоторых принципов работы определённой коллекции операций и данных. Можно просто воспринимать класс, а точнее его конкретные экземпляры как такой хитрый контейнер. Типа такого:

In [1]:
class MyClass(object):
    pass

a = MyClass()
a.name = 'Имя'
a.surname = 'Фамилиaoenuя'
a.middle_name = 'Отчество'

print(a.name, a.surname, a.middle_name)

Имя Фамилиaoenuя Отчество


Выше мы создали объект `a` класса `MyClass` и у него задали три поля `name`, `surname` и `middle_name`. При этом, если мы создадим ещё один объект класса `MyClass`, этих полей у него не будет &mdash; они будут только у объекта `a`. Если мы хотим, чтобы у класса были сразу как-то инициированы какие-то поля, то эти операции надо засунуть в конструктор:

In [2]:
class MyClass(object):
    def __init__(self):  # эта функция вызывается когда мы пишем MyClass()
        self.name = 'Имя'
        self.surname = 'Фамилия'
        self.middle_name = 'Отчество'
        
    def __str__(self):
        return "%s %s. %s" % (
            self.name, self.middle_name[0], self.surname
        )
    
    def rename(sosisochka, newname):
        sosisochka.name = newname
        

In [3]:
a = MyClass()
print(a)

Имя О. Фамилия


&laquo;Как можно легко видеть&raquo;, функция `__str__` вызывается, когда мы объект кастуем к строке, либо неявно через `print` либо через `str(a)`, `f"{a}"` или `"%s" % a` .

In [4]:
b = MyClass()
b.surname = "Онегин"
print(b)

Имя О. Онегин


In [5]:
b.rename("Евгений")
print(b)

Евгений О. Онегин


Вот ещё один, чуть более навороченный пример класса с особым композитным полем `fullname`, которое имеет индивидуальный сеттер, что позволяет хитро задавать сразу три поля объекта одной операцией.

In [6]:
class MyClass(object):
    def __init__(self,
                 name='unknown',
                 surname='unknown',
                 middle_name='unknown'
    ):
        self.name = name
        self.surname = surname
        self.middle_name = middle_name
        
    def __str__(self):
        return self.fullname
    
    def fullname_prosto_tak(self):
        return "AAAA %s %s. %s" % (
            self.name.title(),
            self.middle_name.title()[0],
            self.surname.title()
        )
    
    @property
    def fullname(self):
        return "%s %s. %s" % (
            self.name.title(),
            self.middle_name.title()[0],
            self.surname.title()
        )
    
    @fullname.setter
    def fullname(self, value):
        self.name, self.middle_name, self.surname = value.split(' ', 2)

Часть вашего задания к этой тетрадке &mdash; выяснить что делает в последней строчке у `split` цифра 2.

In [7]:
a = MyClass()
a.fullname

'Unknown U. Unknown'

In [8]:
a = MyClass()
print(a)
a.fullname = "Игорь Юрьевич Мосягин"
print(a)
print(a.surname)

Unknown U. Unknown
Игорь Ю. Мосягин
Мосягин


У класса могут быть специальные непривязанные к конкретным инстансам методы, которые помечаются как `@classmethod` 

In [9]:
class MyClass(object):
    _secret = None
    
    @classmethod
    def hello(cls):
        return "Дратути"
    
    @classmethod
    def init_with_secret(cls, secret):
        s = MyClass()
        s._secret = secret
        return s
    
    def __str__(self):
        return f'{self.hello()}. Секрет: {self._secret}'

В этом примере мы создали класс с "секретным" полем, и не определяли у класса конструктор, но зато определили дополнительный метод который выставляет нужные поля классу.

In [10]:
x = MyClass.init_with_secret(4205)
print(x)

Дратути. Секрет: 4205


In [11]:
unbound_hello = MyClass.hello()
print(unbound_hello)

Дратути


А, да, и обратите внимание что первый аргумент в методах это всегда `self`, а у `classmethods` это `cls`. Это, собственно, текущий объект. Ничто не мешает называть эти первые позиционные аргументы по-другому, но хорошим тоном считается давать им именно такие названия. Считайте это особенностью языка.

А вообще, вот эти все конструкции с символом собачки над определениями функций называются декораторами, и их можно использовать, чтобы менять поведение функции. По сути, когда вы видите конструкцию

```python
@mydecorator
def myfunction(a, b, c):
    ...
```

То надо понимать что в этом месте результат вызова функции `myfunction` будет использован как-то декоратором `@mydecorator`. Вообще декораторы это паттерн аспектно-ориентированного программирования и их, конечно, можно (и часто нужно) писать свои. Правда выглядит немного замороченно:

In [12]:
def my_decorator(f):
    def inner_decorator(*args, **kw):
        print("-- ENTER --")
        f(*args, **kw)
        print("-- EXIT --")
    return inner_decorator

def my_second_decorator(f):
    def inner_deco(*args, **kw):
        print(" ENTER ".center(11, "*"))
        f(*args, **kw)
        print(" EXIT ".center(11, "*"))
    return inner_deco

@my_decorator
@my_second_decorator
def hello():
    print("Hello there")
    
hello()

-- ENTER --
** ENTER **
Hello there
*** EXIT **
-- EXIT --


А если хочется совсем заморочиться, то можно написать вообще класс, объекты которого будут декораторами с какими-то дополнительно определёнными полями, влияющими на поведение. Типа такого:

In [13]:
# Class Decorator
class MyDecorator(object):
    def __init__(self, before, after):
        self.before = before
        self.after = after
        
    def __call__(self, f):
        def inner_f(*args, **kw):
            print(self.before.center(11, "*"))
            f(*args, **kw)
            print(self.after.center(11, "*"))
        return inner_f
    
mydecoratorobject = MyDecorator('Hello', 'Goodbye')

@mydecoratorobject
def f():
    print('Inside Function')
    
f()
    
print('-----')
    
@mydecoratorobject
@MyDecorator('Что-то', 'Другое')
def g():
    print('Inside Another Function')

g()


***Hello***
Inside Function
**Goodbye**
-----
***Hello***
***Что-то**
Inside Another Function
***Другое**
**Goodbye**


А вообще, декораторы бывают ещё и на классы... Но это совсем уже чёрная магия.

---

## Домашнее задание на почитать: Standard library

Речь про описание [стандартной библиотеки](https://docs.python.org/3/library/index.html). Стандартная библиотека большая, есть в любом дистрибутиве питона, и очень полезно уметь в ней ориентироваться.

И вообще все первые двадцать глав из мануала про python стоит прочитать.

Но в прииинцие, это &mdash; справочник, и можно пропустить при прочтении части которые вы вдруг не понимаете сейчас.

При первом прочтении, на мой взгляд, стоит ограничиться следующими разделами:

* 1
* 2
* 3
* 4.1, 4.2, 4.3, 4.5, 4.6, 4.7, 4.9, 4.10
* 8.1, 8.2, 8.3
* 9.1, 9.2, 9.4, 9.6
* 10
* 11.2, 11.6, 11.10
* 12.1, 12,4, 12.6
* 14.1, 14,2
* 24.1
* 25
* 27.5

Они во многом будут повторять то, что мы уже прошли, но должны ещё дополнить ваши возможности. Ниже несколько примеров использования различных функций модулей стандартной библиотеки

In [14]:
import os
print(dir(os))

['CLD_CONTINUED', 'CLD_DUMPED', 'CLD_EXITED', 'CLD_TRAPPED', 'DirEntry', 'EX_CANTCREAT', 'EX_CONFIG', 'EX_DATAERR', 'EX_IOERR', 'EX_NOHOST', 'EX_NOINPUT', 'EX_NOPERM', 'EX_NOUSER', 'EX_OK', 'EX_OSERR', 'EX_OSFILE', 'EX_PROTOCOL', 'EX_SOFTWARE', 'EX_TEMPFAIL', 'EX_UNAVAILABLE', 'EX_USAGE', 'F_LOCK', 'F_OK', 'F_TEST', 'F_TLOCK', 'F_ULOCK', 'GRND_NONBLOCK', 'GRND_RANDOM', 'MutableMapping', 'NGROUPS_MAX', 'O_ACCMODE', 'O_APPEND', 'O_ASYNC', 'O_CLOEXEC', 'O_CREAT', 'O_DIRECT', 'O_DIRECTORY', 'O_DSYNC', 'O_EXCL', 'O_LARGEFILE', 'O_NDELAY', 'O_NOATIME', 'O_NOCTTY', 'O_NOFOLLOW', 'O_NONBLOCK', 'O_PATH', 'O_RDONLY', 'O_RDWR', 'O_RSYNC', 'O_SYNC', 'O_TMPFILE', 'O_TRUNC', 'O_WRONLY', 'POSIX_FADV_DONTNEED', 'POSIX_FADV_NOREUSE', 'POSIX_FADV_NORMAL', 'POSIX_FADV_RANDOM', 'POSIX_FADV_SEQUENTIAL', 'POSIX_FADV_WILLNEED', 'PRIO_PGRP', 'PRIO_PROCESS', 'PRIO_USER', 'P_ALL', 'P_NOWAIT', 'P_NOWAITO', 'P_PGID', 'P_PID', 'P_WAIT', 'PathLike', 'RTLD_DEEPBIND', 'RTLD_GLOBAL', 'RTLD_LAZY', 'RTLD_LOCAL', 'RTLD_N

In [15]:
print(os.listdir('./'))

['some_file.txt', 'L1.2--Exceptions-Functions.ipynb', 'L1.4--Classes-decorators-stdlib.ipynb', 'L1.1--basics.pdf', 'L1.3--Lambda-map-filter-files.ipynb', 'img', 'L1.1--basics.ipynb', '.ipynb_checkpoints']


In [16]:
from os import listdir
print(listdir('.'))

['some_file.txt', 'L1.2--Exceptions-Functions.ipynb', 'L1.4--Classes-decorators-stdlib.ipynb', 'L1.1--basics.pdf', 'L1.3--Lambda-map-filter-files.ipynb', 'img', 'L1.1--basics.ipynb', '.ipynb_checkpoints']


В библиотеке есть всякие полезности. Помните мы писали вот такой примерно код когда считали частоты?

In [17]:
d = {}

if "ключик" in d:
    d["ключик"].append("сосисочка")
else:
    d["ключик"] = []
    d["ключик"].append("сосисочка")
print(d)

{'ключик': ['сосисочка']}


Его можно заменить на более простой с помощью `defaultdict`, в конструктор которому передаётся функция  

In [18]:
from collections import defaultdict

d = defaultdict(list)

d["ключик"].append("сосисочка") 
print(d)

defaultdict(<class 'list'>, {'ключик': ['сосисочка']})


Один из самых популярных модулей называется `datetime`, и содержит в себе всякие операции с датами 

In [19]:
from datetime import datetime as dt

print(dt.now())

2018-04-07 01:19:30.107367


In [20]:
print(dt(2020, 12, 5) - dt.now())

972 days, 22:40:29.500347


In [21]:
wait_time = (dt(2018, 10, 29, hour=22, minute=0) - dt.now())
print(f"Осталось {wait_time} до 29 октября")
print(f"Или, если вам так удобнее, {wait_time.total_seconds()}")

Осталось 205 days, 20:40:28.972915 до 29 октября
Или, если вам так удобнее, 17786428.972915


Ну и на закрепление давайте напишем простую программку.

## Exercise 1.4.0

Сделать функцию, которая спрашивает пользователя дату рождения и говорит, сколько дней он прожил по текущий момент. 

_Подсказка: можно сделать несколькими способами, как обычно. Последние несколько ячеек с кодом содержат всё что вам нужно знать про `datetime` чтобы сделать это задание. Если не хотите возиться с разбором того, что ввёл пользователь, просто спросите у него три раза: сначала год рождения, потом месяц, потом дату._

In [22]:
def tell_user_days():
    days = 999999
    return f"Вы живёте {days} дней"

tell_user_days()

'Вы живёте 999999 дней'

# Ну и теперь ещё задачки

Так как мы по сути обсудили все особенности синтаксиса, предлагается решить несколько околоматематических задачек с использованием питона. Задачки ниже это модифицированные задачки с ресурса [Project Euler](http://projecteuler.net), который я дичайще вам рекомендую.

## Exercise 1.4.1. А что 15?

Если мы перечислим все натуральные числа меньше 10, которые делятся на 3 или 5, то это 3, 5, 6, и 9. Их сумма равна 23.  

Напишите кусок кода который посчитает напрямую сумму всех натуральных чисел меньше 1000, которые делятся на 3 или 5.

_Подсказка: обратите внимание на название задачи_

## Exercise 1.4.2. Фибоначчи раз

Каждый элемент последовательности Фибоначчи равен сумме двух предыдущих. Если мы задаём первые два элемента как 1 и 2, то первые десять будут такие:

1, 2, 3, 5, 8, 13, 21, 34, 55, 89, ...

Во-первых, напишите функцию которая даст вам N-ый элемент последовательности.
А во-вторых, найдите сумму всех **чётных** элементов последовательности фибоначчи меньше 400000.

## Exercise 1.4.3. Фибоначчи два

Рассмотрим ещё раз последновательность Фибоначчи. Можно сказать что она определена так:

$F_n = F_{n-1} + F_{n-2}$, при этом $F_1 = 1$ и $F_2 = 1$ (определение нарочно отличается от определения в прошлой задаче).

Первые 12 элементов последовательности:
$F_1 = 1$

$F_2 = 1$

$F_3 = 2$

$F_4 = 3$

$F_5 = 5$

$F_6 = 8$

$F_7 = 13$

$F_8 = 21$

$F_9 = 34$

$F_{10} = 55$

$F_{11} = 89$

$F_{12} = 144$

Получается, индекс первого трёхзначного элемента равен 12. А какой будет индекс первого стозначного элемента последовательности Фибоначчи? 

## Exercise 1.4.4. Простые числа

Первые шесть простых чисел это 2, 3, 5, 7, 11 и 13. Шестое простое число равно 13.

Чему равно 1001 простое число?

_Подсказка: подумайте какими признаками обладает простое число. Можете почитать про решето Эратосфена. А можете не читать_

## Exercise 1.4.5. Складываем!

$n!$ это $n \times (n - 1) \times (n - 2) \times \ldots \times 3 \times 2 \times 1$

Например, $10! = 10 \times 9 \times 8 \times 7 \times 6 \times 5 \times 4 \times 3 \times 2 \times 1 = 3628800$, при этом сумма цифр в записи числа $10!$ будет $3 + 6 + 2 + 8 + 8 + 0 + 0 = 27$.

Найдите сумму цифр в записи числа $100!$

## Exercise 1.4.5. Sudoku отображатель

Это скорее на десерт и расслабуху. Можете на досуге, кстати, почитать как работают алгоритмы, которые решают Судоку. А пока давайте просто научимся их красиво отображать.

Я хочу функцию, которая на вход получает всю загадку судоку одной строчкой слева-направо сверху-вниз, типа такого вот <br/>
```
........1.......23..4..5......1.........3.6....7...58.....67....1...4...52.......
```

После чего красиво её распечатывает:
```

. . . |. . . |. . 1 
. . . |. . . |. 2 3 
. . 4 |. . 5 |. . . 
------+------+------
. . . |1 . . |. . . 
. . . |. 3 . |6 . . 
. . 7 |. . . |5 8 . 
------+------+------
. . . |. 6 7 |. . . 
. 1 . |. . 4 |. . . 
5 2 . |. . . |. . . 
```

_Подсказка: тут надо повозиться с точностью определения после какого числа резать и что вставлять. Если решать задачу в лоб, то просто посчитайте, после какого числа последовательности идёт какой символ и выводите их. Но это, конечно, не очень изящно. Но зато сработает_

In [31]:
def format_sudoku(sudoku_str):
    for i, e in enumerate(list(sudoku_str)):
        if i % 18 == 4:  # напоминаю, конкретные цифры в примерах с потолка
            print(e, end="\n")
        else:
            print(e, end='')    

In [32]:
format_sudoku("........1.......23..4..5......1.........3.6....7...58.....67....1...4...52.......")

.....
...1.......23..4..
5......1.........3
.6....7...58.....6
7....1...4...52...
....

Всё, если сделаете все эти задачки сами, то на этом, я думаю, можете считать себя молодцом и дальше смотреть в будущее нашей с вами программы более уверенно // IM